In [ ]:
%load_ext lab_black

In [4]:
import nest_asyncio

nest_asyncio.apply()

import asyncio
from playwright.async_api import Playwright, async_playwright

site = "https://moz.com/"
slow_mo = 50
chrome_exe = "/usr/bin/google-chrome"
downloads_path = "/home/ubuntu/Downloads"
user_data = "/home/ubuntu/.config/google-chrome/"
auth_json = "/home/ubuntu/repos/moz/assets/auth.json"


async def main():
    async with async_playwright() as playwright:
        context = await playwright.chromium.launch_persistent_context(
            "session",
            viewport={"width": 1200, "height": 1200},
            downloads_path=downloads_path,
            executable_path=chrome_exe,
            accept_downloads=True,
            channel="chrome",
            slow_mo=slow_mo,
            headless=False,
        )
        page = await context.new_page()
        await context.storage_state(path=auth_json)
        await page.goto(site)
        
        # Codegen activated
        await page.pause()  # Edit this line in for codegen and out for automation.
        
        # -- BEGIN CODEGEN LINES --
        await page.get_by_role("button", name="Why Moz").click()
        await page.get_by_role("button", name="Free SEO Tools").click()
        await page.get_by_text("Link Explorer").nth(1).click()
        # -- END CODEGEN LINES --

        # When done, close the browser.
        await asyncio.sleep(5)
        await context.close()


async def run_main():
    await main()


try:
    asyncio.run(run_main())
except:
    loop = asyncio.get_event_loop()
    loop.run_until_complete(run_main())

Error: Target page, context or browser has been closed